# Deploy machine learning models to Azure

description: (preview) deploy your machine learning or deep learning model as a web service in the Azure cloud.

# Connect to your workspace

In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Workspace.create(name='chrjia-eastus2euap2', subscription_id='13e50845-67bc-4ac5-94db-48d493a6d9e8', resource_group='chrjia-rg')

# Register your model

A registered model is a logical container stored in the cloud, containing all files located at `model_path`, which is associated with a version number and other metadata.



## Register a model from a local file

You can register a model by providing the local path of the model. You can provide the path of either a folder or a single file on your local machine.

In [3]:
import urllib.request
from azureml.core.model import Model
# Download model
urllib.request.urlretrieve("https://aka.ms/bidaf-9-model", 'model.onnx')

# Register model
model = Model.register(ws, model_name='bidaf_onnx', model_path='./model.onnx')

KeyboardInterrupt: 

## Register a model from an Azure ML training run

When you use the SDK to train a model, you can receive either a Run object or an AutoMLRun object, depending on how you trained the model. Each object can be used to register a model created by an experiment run.

- Register a model from an `azureml.core.Run` object

In [12]:
from azureml.core import Run as run

model = run.register_model(model_name='bidaf_onnx',
                           tags={'area': 'qna'},
                           model_path='outputs/model.onnx')
print(model.name, model.id, model.version, sep='\t')

TypeError: register_model() missing 1 required positional argument: 'self'

- Register a model from an `azureml.train.automl.run.AutoMLRun` object

In [ ]:
 description = 'My AutoML Model'
    model = run.register_model(description = description,
                               tags={'area': 'qna'})

    print(run.model_id)

# Define an inference configuration

The inference configuration below specifies that the machine learning deployment will use the file echo_score.py in the ./source_dir directory to process incoming requests and that it will use the Docker image with the Python packages specified in the project_environment environment.

In [3]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name='azmlenv')
inference_config = InferenceConfig(environment=env, source_directory='./src', entry_script='./echo_score.py')

## Define a deployment configuration

In [4]:
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=6789)

## Deploy your machine learning model

A deployment configuration specifies the amount of memory and cores to reserve for your webservice will require in order to run, as well as configuration details of the underlying webservice. For example, a deployment configuration lets you specify that your service needs 2 gigabytes of memory, 2 CPU cores, 1 GPU core, and that you want to enable autoscaling.

The options available for a deployment configuration differ depending on the compute target you choose. In a local deployment, all you can specify is which port your webservice will be served on.

In [1]:
service = Model.deploy(ws, "myservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)

NameError: name 'Model' is not defined

In [ ]:
print(service.get_logs())

## Call into your model

In [ ]:
import requests

uri = service.scoring_uri
requests.get('http://localhost:6789')
headers = {'Content-Type': 'application/json'}
data = {"query": "What color is the fox", "context": "The quick brown fox jumped over the lazy dog."}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

Notice the use of the AZUREML_MODEL_DIR environment variable to locate your registered model. Now that you've added some pip packages, you also need to update your inference configuration to add in those additional packages

In [ ]:
env = Environment(name='myenv')
python_packages = ['nltk', 'numpy', 'onnxruntime']
for package in python_packages:
    env.python.conda_dependencies.add_pip_package(package)

inf_config = InferenceConfig(environment=env, source_directory='./source_dir', entry_script='./score.py')

## Deploy again and call your service

In [ ]:
service = Model.deploy(ws, "myservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)

In [ ]:
print(service.get_logs())

Then ensure you can send a post request to the service:

In [ ]:
import requests

uri = service.scoring_uri

headers = {'Content-Type': 'application/json'}
data = {"query": "What color is the fox", "context": "The quick brown fox jumped over the lazy dog."}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

## Re-deploy to cloud

Once you've confirmed your service works locally and chosen a remote compute target, you are ready to deploy to the cloud.

Change your deploy configuration to correspond to the compute target you've chosen, in this case Azure Container Instances.

In [ ]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 0.5, memory_gb = 1)

Deploy your service again

In [ ]:
service = Model.deploy(ws, "myservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)

In [ ]:
print(service.get_logs())

## Call your remote webservice

When you deploy remotely, you may have key authentication enabled. The example below shows how to get your service key with Python in order to make an inference request.

In [ ]:
import requests
import json
from azureml.core import Webservice

service = Webservice(workspace=ws, name='myservice')
scoring_uri = service.scoring_uri

# If the service is authenticated, set the key or token
primary_key, _ = service.get_keys()

# Set the appropriate headers
headers = {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response and logs
data = {"query": "What color is the fox", "context": "The quick brown fox jumped over the lazy dog."}
data = json.dumps(data)
resp = requests.post(scoring_uri, data=data, headers=headers)
print(resp.text)

In [ ]:
print(service.get_logs())

## Delete resources

In [ ]:
service.delete()
model.delete()

## Next Steps

Try reading [our documentation](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python)